In [3]:
import obspy
from obspy import UTCDateTime as utc
from obspy.clients.fdsn import Client
client = Client('IRIS')
import numpy as np
import os

In [4]:
def process(st, buffer, freq, max_target_frequency): #freq is the original sampling frequency
    st.merge(fill_value='interpolate')
    tr = st[0].copy()

In [7]:
#creating variables to download data
starttime = utc('2023-07-9T00:00:00')
endtime = starttime + 100 * (60*60*24)

#also add a buffer to both ends to chop off once the data has been filtered
#and downsampled, kind of arbitrary length, 5% of a day (default ObsPy taper length)
buffer = 60*60*24*0.05 #seconds

net = 'AV'
sta = 'SSLS'
loc = '*' #wildcard, generally don't care about location code
cha = 'BHN' #horizontal component, as used in Zali et al

#create folder for numpy streams to go into and initialize filepath
!mkdir data
!mkdir data/raw
filepath = os.getcwd() + '/data/raw/'

#create arrays to save dates
dates = np.array([])

print(f'Downloading data from {starttime} to {endtime}')

#download the data piecemeal, here by day
for day in range(100):
    tr_length = 24*60*60

    #actually downloading
    st = client.get_waveforms(network=net,
                     station=sta,
                     location=loc,
                     channel=cha,
                     starttime=starttime-buffer,
                     endtime=starttime+buffer+tr_length)

    #instrument sampling rate (hz)
    freq = st[0].stats.sampling_rate

    #merge traces within stream, linearly interpolating any gaps
    st.merge(fill_value='interpolate')
    
    #generate filename, day number in front for convenience of reading in
    name = str(day+1)+'_sitkin.mseed'
    
    #save data as mseed, standard for storing seismic data. Preserves metadata and time series info
    st.write(filepath+name, format='MSEED')

    #adding date
    dates = np.append(dates, starttime.date)

    starttime += tr_length

#save dates list for future use
np.save(filepath+'date_list.csv', dates)

print(f'data download complete, saved to {filepath}')

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/raw’: File exists
data download complete, saved to /home/jovyan/data/raw/


In [8]:
# display some data from the mseed file
print(f'Example data from {name}')
print(st[0].stats)
print()

# display the numpy array
print(f'Example data from {name}')
print(st[0].data)
print()

Example data from 100_sitkin.mseed
               network: AV
               station: SSLS
              location: 
               channel: BHN
             starttime: 2023-10-15T22:48:00.000000Z
               endtime: 2023-10-17T01:12:00.000000Z
         sampling_rate: 50.0
                 delta: 0.02
                  npts: 4752001
                 calib: 1.0
_fdsnws_dataselect_url: http://service.iris.edu/fdsnws/dataselect/1/query
               _format: MSEED
                 mseed: AttribDict({'dataquality': 'M', 'number_of_records': 13525, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 6924800})
            processing: ['ObsPy 1.4.1: trim(endtime=UTCDateTime(2023, 10, 17, 1, 12)::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2023, 10, 15, 22, 48))']

Example data from 100_sitkin.mseed
[-1221 -1358 -1385 ...   655   658   565]

